# Images and Metadata

Loading and saving volumetric data like image or masks is a key component of any medical image analysis pipeline. A major
difference to loading a JPG or PNG image is the associated metadata, such as information about the physical spacing between
pixels and the orientation of the volume relative to the scanner, but also patient data etc. ITK combines pixel data and
metadata into a single class so that the pixel data and the corresponding metadata are always represented as a single object.
However, Tiger follows a different philosophy: pixel data is represented as numpy arrays and the corresponding metadata is
a separate variable, usually an instance of the `tiger.io.ImageMetadata` class. Functions within Tiger therefore
often ask for an "image" and a "header" object.

## Reading medical image formats

Most common data formats are supported (mha/mhd, nii/nii.gz, nrrd, tif, ...).

In [ ]:
from tiger.io import read_image

image, header = read_image("/path/to/image.mha")
mask, _ = read_image("/path/to/mask.mha")

The order of the axes of the pixel data array will be as intuitively expected (x, y, z) instead of reversed (z, y, x).

DICOM images can be read either from a list of files or from a directory. In both cases, the slices will be sorted according
to the standard DICOM coordinate system.

In [ ]:
from tiger.io import read_dicom, DicomReader

# If there are multiple series, only one of them will be returned
image, header = read_dicom(["/path/to/slice1.dcm", "/path/to/slice2.dcm"])

# Iterate over all series in a directory
reader = DicomReader()
for image, header in reader("/path/to/dicom"):
    pass

## Metadata

The `tiger.io.ImageMetadata` class can store any kind of metadata, but there are only four items that will always
be present and for which the class ensures that values are plausible. These are related to the physical coordinate space
of the image:

* Number of dimensions, i.e., whether the image is 2D, 3D, or 4D (ndim)
* Pixel spacing (spacing)
* Coordinates of the first pixel (0, 0, ...) in the world coordinate space (origin)
* Orientation of the pixel data relative to the world coordinate space as a direction cosine matrix (direction)

The class provides methods for transforming coordinates from indices to world coordinates and vice versa.

## Writing medical image formats

Saving images or masks to disk is simple. The `tiger.io.write_image` function will by default automatically make sure
that the parent directories are created if necessary and that the image is stored in compressed form. These things can be
disabled, but the general idea is that saving image data should be a painless experience that requires only a single command.

In [ ]:
from tiger.io import write_image

write_image("/path/to/image.mha", image, header)
write_image("/path/to/mask.nii.gz", mask)  # header is optional, will default to unit spacing etc.


## Converting to/from SimpleITK

The Tiger combination of numpy array and metadata object can be converted into a single SimpleITK image object using the
`tiger.io.image_to_sitk`. Likewise, SimpleITK images can be converted using `tiger.io.image_from_sitk`.

Currently, additional metadata is not completely preserved though.